# Calculate Angular Scale of BAO

A notebook for calculating the scale of the BAO in CMB. To do the integrations, we make use of `scipy.integrate.quad`   
Comoving sound horizon s = $\int^{\infty}_{z_{rec}} \frac{c_{s}dz}{H(z)}$    
Comoving distance r = $\int_{0}^{z_{rec}} \frac{c dz}{H(z)}$   
Scale of the acoustic peak $l = S[r(z)]/s$   

In [2]:
%matplotlib inline
import numpy as np
from scipy import integrate
from scipy.constants import *
import matplotlib.pyplot as plt

In [3]:
h = 1.0
omega_r = 

In [38]:
def invadot(a, om_m=0.3, om_L=0.0, h=.696):
    om_r = 4.165e-5*h**-2 # T0 = 2.72528K
    answ = 1/np.sqrt(om_r/(a * a) + om_m / a\
            + om_L*a*a + (1.0-om_r-om_m-om_L))
    return answ

def invaadot(a, om_m=0.3, om_L=0.0, h=.696):
    om_r = 4.165e-5*h**-2 # T0 = 2.72528K
    answ = 1/np.sqrt(om_r/(a * a) + om_m / a\
            + om_L*a*a + (1.0-om_r-om_m-om_L))
    return answ/a



class cosmology(object):
    '''
       cosmology
    '''    
    def __init__(self, om_m=1.0, om_L=0.0, h=.696):
        self.om_m = om_m
        self.om_L = om_L
        self.h    = h
        self.om_r = 4.165e-5*h**-2 # T0 = 2.72528K
        self.Tyr  = 9.778/h
        self.Mpc  = c*1.e-5/h
    
    def zage_Gyr(self, z):
        az = 1 / (1+z)
        answ,_ = integrate.quad(invadot, 0, az,
                               args=(self.om_m, self.om_L, self.h))
        return answ * self.Tyr
    
    def age_Gyr_now(self):
        answ,_ = integrate.quad(invadot, 0, 1,
                               args=(self.om_m, self.om_L, self.h))
        return answ * self.Tyr
    
    def DCMR(self, z):
        az = 1 / (1+z)
        answ,_ = integrate.quad(invaadot, az, 1,
                               args=(self.om_m, self.om_L, self.h))
        return answ * self.Mpc
    
    def DA(self, z):
        az = 1 / (1+z)
        r,_ = integrate.quad(invaadot, az, 1,
                               args=(self.om_m, self.om_L, self.h))
        r *= self.Mpc
        om_k = (1.0-self.om_r-self.om_m-self.om_L)
        if om_k != 0.0:DHabsk = self.Mpc/np.sqrt(np.abs(om_k))
        if om_k > 0.0:
            Sr = DHabsk * np.sinh(r/DHabsk)
        elif om_k < 0.0:
            Sr = DHabsk * np.sin(r/DHabsk)
        else:
            Sr = r
        return Sr*az
    
    def DL(self, z):
        az = 1 / (1+z)
        da = self.DA(z)
        return da / (az * az)
        
    
    



# def invH(z, om_m=0.3, om_L=0.0, h=.696):
#     om_r = 4.165e-5*h**-2 # T0 = 2.72528K
#     answ = 1./(np.sqrt(om_r*(1.+z)**4 + om_m*(1.+z)**3+\
#           om_L+(1.0-om_r-om_m-om_L)*(1+z)**2))
#     return answ

# def zage(z, om_m, om_L, h=.696):
#     Tyr = 9.778 # 1/h to Gyr
#     az = 1 / (1+z)
#     answ,_ = integrate.quad(invadot, 0, az,
#                                args=(om_m, om_L, h))
#     return answ*(Tyr/h)


# def sound_horizon(om_r, om_m, om_L=0.0, h=1.0, z_rec=1000., 
#                   funct=H, verbose=False):
#     """
#        computes the sound horizon for a given cosmology
#     """
#     DH = c*1.e-5/h
#     answ, err = integrate.quad(funct, z_rec, np.inf,
#                                args=(om_r, om_m, om_L))
#     answ *= DH/np.sqrt(3.)
#     if verbose:
#         print("for h {}, om_r {}, om_m {}, & om_L {}\
#         the sound horizon is : {:.1f} Mpc"\
#         .format(h, om_r, om_m, om_L, answ))
#     return answ


# def comov_dist(om_r, om_m, om_L=0.0, h=1.0, z_rec=1000., 
#                funct=H, verbose=False):
#     """
#        comoving diameter distance using Quadpack to do the integral
#     """
#     DH = c*1.e-5/h
#     answ, err = integrate.quad(funct, 0.0, z_rec, args=(om_r, om_m, om_L))
#     answ *= DH # 3000/h Mpc
#     if verbose:
#         print("for h {}, om_r {}, om_m {}, & om_L {} \
#         the comov. dist. is : {:.1f} Mpc"\
#         .format(h, om_r, om_m, om_L, answ))
#     return answ


# def metric_dist(om_r, om_m, om_L=0.0, h=1.0, z_rec=1000., 
#                 funct=H, verbose=False):
#     """
#         metric distance ie. S[r(z)] depends on the curvature
#     """
#     DH = c*1.e-5/h
#     om_k = 1.0-om_r-om_m-om_L
#     r = comov_dist(om_r, om_m, om_L=om_L, h=h, z_rec=z_rec, funct=funct)
#     if om_k != 0.0:DHabsk = DH/np.sqrt(np.abs(om_k))
#     if om_k > 0.0:
#         Sr = DHabsk * np.sinh(r/DHabsk)
#     elif om_k < 0.0:
#         Sr = DHabsk * np.sin(r/DHabsk)
#     else:
#         Sr = r
#     if verbose:
#         print("curvature is : ", om_k)
#         print("S[r(z)] is : {:.1f} Mpc".format(Sr))
#     return Sr

# def lacoustic(om_r, om_m, om_L=0.0, h=1.0, z_rec=1000., funct=H, verbose=False):
    
#     Sr = metric_dist(om_r, om_m, om_L=om_L, h=h, verbose=verbose)
#     s = sound_horizon(om_r, om_m, om_L=om_L, h=h, verbose=verbose)
#     lacous = 4.*Sr/s
#     print("l_peak : ", int(lacous))

In [39]:
universe = cosmology(0.286, 0.714, h=.696)

In [53]:
universe.zage_Gyr(0)

13.720706530961097

In [46]:
universe.age_Gyr_now()

13.720706530961097

In [47]:
universe.DCMR(1000)

13937.765156807156

In [51]:
universe.DA(1000)

13.921752268778331

In [52]:
universe.DL(1000)

13949609.695068156